In [1]:
import nivapy3 as nivapy
import nope
import useful_rid_code as rid
import pandas as pd

# Generate overall summary table - 2018

Table 3 of the 2015 report summarises all monitored and modelled loads for the whole of Norway, and for the four OSPAR regions. This notebook reads the output from the processing of the observed data (see [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/rid_working_2016-17.ipynb)) and the modelling using NOPE (see [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/loads_unmonitored_regions_2016.ipynb)) and generates a Word version of Table 3. The code is based on the exploratory work [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/word_data_tables.ipynb).

## 1. Summarise monitoring data

In [2]:
# Read station data
in_xlsx = r'../../../Data/RID_Sites_List_2017-2020.xlsx'
stn_df = pd.read_excel(in_xlsx, sheet_name='RID_All')

# Get just cols of interest and drop duplicates 
# (some sites are in the same regine)
stn_df = stn_df.drop_duplicates(subset=['ospar_region', 'nve_vassdrag_nr'])

# Get catch IDs with calib data
calib_nds = set(stn_df['nve_vassdrag_nr'].values)

# Build network
in_path = r'../../../NOPE/NOPE_Annual_Inputs/nope_input_data_1990.csv'
g, nd_list = nope.build_calib_network(in_path, calib_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df['nve_vassdrag_nr'].isin(ds_nds)]

**Note:** Remember to update the year below.

In [3]:
# Year of interest
year = 2018

In [4]:
# Read data
in_csv = r'../../../Results/Loads_CSVs/loads_and_flows_all_sites_%s.csv' % year
mon_df = pd.read_csv(in_csv)

# Get just the downstream catchments
#mon_df = mon_df[mon_df['station_id'].isin(stn_df['station_id'].values)]

# Group by OSPAR region
mon_df1 = mon_df.groupby(['ospar_region', 'new_rid_group']).sum()

# Totals for Norway
mon_df2 = mon_df.groupby('new_rid_group').sum().reset_index()
mon_df2['ospar_region'] = 'NORWAY'
mon_df2.set_index(['ospar_region', 'new_rid_group'], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split('_')[1] != 'Est']
mon_df = mon_df[cols]
del mon_df['station_id']

# Rename cols to match template
mon_df['Flow rate_1000m3/day'] = mon_df['mean_q_1000m3/day']
del mon_df['mean_q_1000m3/day']

# Units are correct, so remove
mon_df.columns = [i.split('_')[0] for i in mon_df.columns]

mon_df.round(0)

Ag    As   Cd    Cr     Cu    Hg   NH4-N  \
ospar_region        new_rid_group                                              
LOFOTEN-BARENTS SEA rid_135        0.0   1.0  0.0   1.0    7.0   7.0    93.0   
                    rid_20         0.0   3.0  0.0   4.0  117.0   2.0   251.0   
NORTH SEA           rid_135        0.0   3.0  0.0   4.0   22.0  30.0   339.0   
                    rid_20         0.0   1.0  0.0   0.0    2.0   2.0    36.0   
NORWEGIAN SEA2      rid_135        0.0   4.0  0.0   8.0   27.0  28.0   312.0   
                    rid_20         0.0   1.0  0.0   2.0   15.0   0.0    36.0   
SKAGERAK            rid_135        0.0   3.0  0.0   2.0    8.0  12.0   186.0   
                    rid_20         0.0   8.0  1.0   9.0   50.0   0.0   683.0   
NORWAY              rid_135        0.0  11.0  1.0  15.0   65.0  77.0   931.0   
                    rid_20         0.0  13.0  1.0  15.0  185.0   4.0  1006.0   

                                     NO3-N     Ni  PO4-P    Pb       SPM  \
ospar_region        new_rid_group                                          
LOFOTEN-BARENTS SEA rid_135          304.0    5.0   17.0   0.0   15497.0   
                    rid_20           629.0  113.0   40.0   2.0   33953.0   
NORTH SEA           rid_135         6795.0   12.0   81.0   7.0   73661.0   
                    rid_20          1157.0    2.0   14.0   1.0    6919.0   
NORWEGIAN SEA2      rid_135         2578.0   21.0   88.0   2.0  116129.0   
                    rid_20          1029.0    6.0   14.0   1.0   21312.0   
SKAGERAK            rid_135         2660.0    5.0   29.0   4.0   25287.0   
                    rid_20         14752.0   27.0  269.0   9.0  216067.0   
NORWAY              rid_135        12337.0   42.0  216.0  14.0  230573.0   
                    rid_20         17566.0  148.0  337.0  12.0  278250.0   

                                       SiO2       TOC     TOTN   TOTP     Zn  \
ospar_region        new_rid_group                                              
LOFOTEN-BARENTS SEA rid_135         30332.0   28951.0   1423.0   31.0    8.0   
                    rid_20          88843.0   51416.0   3019.0  111.0   32.0   
NORTH SEA           rid_135         65653.0   73647.0  11939.0  231.0   83.0   
                    rid_20           7154.0    8950.0   1750.0   34.0    9.0   
NORWEGIAN SEA2      rid_135         74533.0   78821.0   7591.0  206.0   66.0   
                    rid_20          23745.0   24219.0   2054.0   45.0   29.0   
SKAGERAK            rid_135         27735.0   59770.0   5461.0  114.0   54.0   
                    rid_20         150386.0  171975.0  23004.0  492.0  167.0   
NORWAY              rid_135        198253.0  241189.0  26413.0  581.0  211.0   
                    rid_20         270129.0  256559.0  29827.0  682.0  237.0   

                                   Flow rate  
ospar_region        new_rid_group             
LOFOTEN-BARENTS SEA rid_135          37148.0  
                    rid_20           48751.0  
NORTH SEA           rid_135         147816.0  
                    rid_20           16495.0  
NORWEGIAN SEA2      rid_135         118192.0  
                    rid_20           31949.0  
SKAGERAK            rid_135          33903.0  
                    rid_20          140241.0  
NORWAY              rid_135         337059.0  
                    rid_20          237436.0

## 2. Data for unmonitored areas

**Note:** Remember to update the year below.

In [5]:
# Read data
in_csv = r'../../../Results/Unmon_loads/unmon_loads_%s.csv' % year
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace('RENSEANLEGG', 'sew') for i in umon_df.columns]
umon_df.columns = [i.replace('INDUSTRI', 'ind') for i in umon_df.columns]
umon_df.columns = [i.replace('_tonn', '') for i in umon_df.columns]
umon_df.columns = [i.replace('AQUAKULTUR', 'fish') for i in umon_df.columns]

# Convert Hg to kgs
umon_df['sew_Hg'] = umon_df['sew_Hg']*1000
umon_df['ind_Hg'] = umon_df['ind_Hg']*1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,448260.0,13095.0,1099.0,2582.0,262.0,60256.0,10404.0,36655.0,731.0,659.0,...,12908.0,0.0,0.0,0.0,4.0,13.0,2.0,1.0,8.0,1217.0
LOFOTEN-BARENTS SEA,127960.0,1577.0,176.0,128.0,6.0,14879.0,2574.0,5930.0,116.0,106.0,...,2691.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,301.0
NORTH SEA,163155.0,3551.0,476.0,487.0,95.0,18894.0,3249.0,15224.0,246.0,286.0,...,3739.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,380.0
NORWEGIAN SEA2,146838.0,2687.0,350.0,992.0,113.0,26442.0,4574.0,12480.0,281.0,210.0,...,4584.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,6.0,536.0
SKAGERAK,10308.0,5280.0,97.0,975.0,48.0,40.0,7.0,3021.0,88.0,58.0,...,1894.0,0.0,0.0,0.0,2.0,8.0,1.0,0.0,1.0,1.0


## 3. Create table

The cell below uses the 3-fold division of sites (`'rid_11'` etc.) from the old progreamme structure. To use this, copy the old Word template and make sure that the aggregation in Section 1 is done by `'old_rid_group'`.

In [ ]:
## Create table 3 (old programme structure)
#in_docx = f'../../../Results/Word_Tables/{year + 1}Analysis_{year}Data/Table_3_{year}.docx'
#
#rid.write_word_overall_table(mon_df, umon_df, in_docx)

The cell below uses the 2-fold division of sites (`'rid_20'` and `'rid_135'`) for the 2017-20 progreamme structure. To use this, copy the new Word template and make sure that the aggregation in Section 1 is done by `'new_rid_group'`.

In [6]:
# Create table 3
in_docx = f'../../../Results/Word_Tables/{year + 1}Analysis_{year}Data/Table_3_{year}_new.docx'

rid.write_word_overall_table_2017_20(mon_df, umon_df, in_docx)

Finished.
